### Import relevant libraries

In [1]:
import os
import importlib
import pandas as pd

from nilearn import input_data
from bids import BIDSLayout
from nilearn.interfaces import fmriprep

import process_bids as pb
import process_connectome as pc

importlib.reload(pb)
importlib.reload(pc)

pd.options.mode.chained_assignment = None

C:\Users\smine\anaconda3_new\envs\biotypes\lib\site-packages\nilearn\input_data\__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


### Load input data

In [2]:
dataset_name = 'lightduo'
#raw_path = os.path.abspath('D:\msc-raw')
derivatives_path = os.path.abspath(r'..\..\biotypes\dataset_sample\lightduo-preproc\derivatives\fmriprep')

# Lecture du format BIDS
layout = BIDSLayout(derivatives_path, index_metadata=True, reset_database=False, validate=False,
                    config=["bids", "derivatives"])

layout

C:\Users\smine\anaconda3_new\envs\biotypes\lib\site-packages\bids\layout\validation.py:51: UserWarning: The ability to pass arguments to BIDSLayout that control indexing is likely to be removed in future; possibly as early as PyBIDS 0.14. This includes the `config_filename`, `ignore`, `force_index`, and `index_metadata` arguments. The recommended usage pattern is to initialize a new BIDSLayoutIndexer with these arguments, and pass it to the BIDSLayout via the `indexer` argument.
  warnings.warn("The ability to pass arguments to BIDSLayout that control "


BIDS Layout: ...o-preproc\derivatives\fmriprep | Subjects: 1 | Sessions: 2 | Runs: 1

### Input variables

In [8]:
subjects_id_list = layout.get_subjects()
t_r = float(layout.get_RepetitionTime()[0])
print('Subjects ID:', subjects_id_list)
print('Repetition Time:', t_r)

Subjects ID: ['01']
Repetition Time: 2.0


### Load brain parcellation

In [ ]:
atlas_name = 'CAB-NP'
atlas_path = os.path.abspath(r'..\..\atlas\CAB-NP_volumetric\CAB-NP_volumetric_liberal.nii.gz')
atlas_masker = input_data.NiftiLabelsMasker(atlas_path, standardize=False, smoothing_fwhm=6, low_pass=0.01, t_r=t_r)

### Parallele processing for time series calculation

In [7]:
time_series_info = pd.DataFrame(columns = subjects_id_list, index = ['path','nlevels','atlas'])
time_series_info.loc['atlas',:] = atlas_name

In [8]:
directory = 'timeSeries_files'

sub_dir = os.path.join(directory,dataset_name,atlas_name)
if not os.path.exists(sub_dir):
    os.makedirs(sub_dir)

confounds_name = ['None','CompCor']

In [ ]:
for id in subjects_id_list:
    csv_path = os.path.join(sub_dir ,'id_atlas_timeSeries.csv'.replace('id',id).replace('atlas',atlas_name))

    sessions_sub = pb.get_sessions(layout, id)
    tasks_by_session_sub = pb.get_tasks(layout, id, sessions_sub)
    run_by_task_sub = pb.get_run(layout, id, sessions_sub, tasks_by_session_sub)
    subj_timeseries_df = pc.create_empty_df_timesseries(sessions_sub,tasks_by_session_sub, run_by_task_sub, confounds_name)

    # Calcul sur l'ensemble des nifti
    subj_nii_path = layout.get(subject = id, extension='nii.gz', suffix='bold', scope='derivatives',
                                  return_type='filename')


    subj_CompCor = fmriprep.load_confounds_strategy(subj_nii_path, denoise_strategy='compcor')[0]

    time_series_info.loc['path',id] = csv_path
    time_series_info.loc['nlevels',id] = subj_timeseries_df.columns.nlevels
    subj_timeseries_df = pc.calculate_timeseries(atlas_masker, run_by_task_sub, subj_nii_path, csv_path,
                                                subj_timeseries_df, t_r, subject_confounds_list = [subj_CompCor],
                                                confounds_name= confounds_name)

In [ ]:
info_csv_path = os.path.join(sub_dir,'timeSeries_info.csv')
time_series_info.to_csv(info_csv_path, header= True)
time_series_info

### Subjects caracteristiques

In [ ]:
time_series_info = pd.read_csv(info_csv_path, header = 0, index_col=0)
time_series_info

### Connectomes calculation

In [ ]:
directory = 'connectomes'
sub_dir = os.path.join(directory,dataset_name, atlas_name)

confounds_name.remove('None')

for confound in confounds_name:
    conf_dir = os.path.join(sub_dir,confound)
    if not os.path.exists(conf_dir):
        os.makedirs(conf_dir)

In [ ]:
kind_connect = 'correlation'
vectorize_connect = True
discard_diagonal_connect = True

pc.extract_connectomes(time_series_info, sub_dir, confounds_name, kind=kind_connect, vectorize=vectorize_connect,
                       discard_diagonal=discard_diagonal_connect)
